In [1]:
import numpy as np
from scipy.io import wavfile
import matplotlib.pyplot as plt 
import libfmp.c2
#import subprocess as sp

#Eventuale conversione MP3 -> WAV con ffmpeg
#cmdffmpeg = "./ffmpeg -i A2.mp3 -vn -acodec pcm_s16le -ac 1 -ar 44100 -f wav A2.wav"
#print(cmdffmpeg)
#sp.call(cmdffmpeg) # Esegue il comando ffmpeg

f_sr, data = wavfile.read("Bm.wav")
a = data.T[0]
b = data.T[1]

plt.figure()
plt.grid(True)
plt.plot(a)
plt.xlabel("Time")
plt.ylabel("Intensity PCM 16bit")
plt.title("Traccia 0")

N = 4096
X = np.fft.fft(a, N)
Y = np.abs(X) ** 2
f_ref = 440

In [2]:
def f_pitch(p):
    return 2 ** ((p - 69) / 12) * 440

def pool_pitch(p, Fs, N):
    lower = f_pitch(p - 0.5)
    upper = f_pitch(p + 0.5)
    k = np.arange(N // 2 + 1)
    k_freq = k * Fs / N  # F_coef(k, Fs, N)
    mask = np.logical_and(lower <= k_freq, k_freq < upper)
    return k[mask]

def compute_spec_log_freq(Y, Fs, N):
    Y_LF = np.zeros((128, Y.shape[0]))
    for p in range(128):
        k = pool_pitch(p, Fs, N)
        Y_LF[p] = Y[k].sum(axis=0)
    F_coef_pitch = np.arange(128)    
    return Y_LF, F_coef_pitch

def compute_chromagram(Y_LF):
    C = np.zeros((12, Y_LF.shape[1]))
    p = np.arange(128)
    for c in range(12):
        mask = (p % 12) == c
        C[c, :] = Y_LF[mask, :].sum(axis=0)
    return C

Y_LF, F_coef_pitch = compute_spec_log_freq(Y, f_sr, N)
C = compute_chromagram(Y_LF)

v = []
for i in range(12):
    s = 0
    for j in range(4096):
        s += C[i][j]
    v.append(s)

for i in range(12):
    print(v[i])

print(np.argsort(v))

7612765229251415.0
7092978327654155.0
5.993115661579247e+16
2062133593985006.5
2131769020860149.8
8737824343277584.0
2.14684769767558e+17
1.203981949716319e+16
3438400874783388.5
5713087706140674.0
1901336361432349.5
9.531736018826072e+16
[10  3  4  8  9  1  0  5  7  2 11  6]


In [3]:
temp = np.argsort(v)
print(temp)
bigthree = [temp[11], temp[10], temp[9]]
print(bigthree)

[10  3  4  8  9  1  0  5  7  2 11  6]
[6, 11, 2]


In [4]:
CTT = []
CTT.append([0, 4, 7])
for i in range(1, 12):
    a = 0
    temp1 = CTT[i-1];
    temp2 = []
    for j in range(3):
        a = (temp1[j] + 1) % 12
        temp2.append(a)
    CTT.append(temp2)
CTT.append([0, 3, 7])
for i in range(1,12):
    a = 0
    temp1 = CTT[i+11];
    temp2 = []
    for j in range(3):
        a = (temp1[j] + 1) % 12
        temp2.append(a)
    CTT.append(temp2)
#CTT

In [5]:
def check(a, b):
    if(len(a) != len(b)):
        print("I vettori devono avere stessa dimensione!")
        return False
    count = 0
    for i in range(len(a)):
        for j in range(len(b)):
            if(a[i] == b[j]):
                count += 1
        if(count == len(a)):
            return True
    return False

In [6]:
def match(ctt, bt):
    for i in range(len(ctt)):
        if(check(ctt[i], bt)):
            return(i)
    return -1

In [7]:
match(CTT, bigthree)

23